In [ ]:
# Автоперезагрузка импортов
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys

# Подключаем нужные пути
SRC_DIR = Path('src')
sys.path.append(str(SRC_DIR))

from data_utils import create_dataset_processed, split_dataset
from next_token_dataset import create_loaders

# Очистка датасета и токенизация
n_rows = create_dataset_processed()
print('processed rows:', n_rows)

# Разбиение на выборки train/val/test
n_train, n_val, n_test = split_dataset()
print("splits:", n_train, n_val, n_test)

# Создание Dataset и DataLoader
(dsets, loaders, tokenizer) = create_loaders(
    data_dir='data',
    tokenizer_name='bert-base-multilingual-cased',
    seq_len=32,
    batch_size=64,
    add_eos=True,
)

train_ds, val_ds, test_ds = dsets
train_loader, val_loader, test_loader = loaders

batch = next(iter(train_loader))
print("batch:")
for k, v in batch.items():
    print(k, v.shape)

# Первые токены первой последовательности TODO: убрать
print("batch first tokens:")
print(batch["input_ids"][0][:10])
print(batch["labels"][0][:10]) 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[OK] Создан очищенный датасет /Users/alexeypokrovsky/Documents/Work/Learning/Development/Backend/Python/ai-text-autocomplete/data/dataset_processed.csv, строк: 1600498
processed rows: 1600498
[OK] Созданы тренировочная, валидационная, тестовая выборки: 1280398, Val: 160050, Test: 160050
splits: 1280398 160050 160050
[OK] Созданы Dataset и DataLoader для обучения модели.
batch:
input_ids torch.Size([64, 32])
labels torch.Size([64, 32])
batch first tokens:
tensor([10228, 10921, 11951, 10435, 10590, 11155, 10944,   189, 42852, 10271])
tensor([10921, 11951, 10435, 10590, 11155, 10944,   189, 42852, 10271, 10114])
